# Ejercicio: Paralelización de un Proceso de Cálculo Intensivo

Para ilustrar la paralelización de un proceso de cálculo intensivo, vamos a considerar el problema de calcular los números primos dentro de un rango dado utilizando el método del **Criba de Eratóstenes**. Este algoritmo es un buen candidato para la paralelización porque el proceso de marcar los múltiplos de cada primo puede realizarse de manera independiente para diferentes segmentos del rango.

### Algoritmo Secuencial del Criba de Eratóstenes

Primero, presentamos una versión básica y secuencial del Criba de Eratóstenes para encontrar números primos hasta un número \(n\).

In [ ]:
def sieve_of_eratosthenes(n):
    prime = [True for _ in range(n+1)]
    p = 2
    while (p * p <= n):
        if (prime[p] == True):
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1
    primes = [p for p in range(2, n) if prime[p]]
    return primes

### Paralelización del Proceso

Para paralelizar este algoritmo, podemos dividir el rango de números en segmentos más pequeños y asignar cada segmento a un hilo diferente para marcar los múltiplos de los números primos. Python ofrece varias maneras de implementar paralelismo, una de las cuales es utilizando el módulo `concurrent.futures` para la ejecución en paralelo.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import math

def mark_multiples(prime, start, end, p):
    for i in range(start, end, p):
        prime[i] = False

def parallel_sieve_of_eratosthenes(n):
    prime = [True for _ in range(n+1)]
    segment_size = math.ceil(math.sqrt(n))  # Determina el tamaño de cada segmento
    with ThreadPoolExecutor() as executor:
        p = 2
        while (p * p <= n):
            if (prime[p] == True):
                start_index = max(p * p, 2*p)  # Asegura comenzar en el primer múltiplo dentro del segmento
                futures = [executor.submit(mark_multiples, prime, i, min(i+segment_size, n+1), p) for i in range(start_index, n+1, segment_size)]
                # Espera a que todos los hilos completen su trabajo
                [f.result() for f in futures]
            p += 1
    primes = [p for p in range(2, n) if prime[p]]
    return primes

### Análisis de la Paralelización

- **Eficiencia**: La paralelización puede reducir significativamente el tiempo de ejecución para grandes valores de `n` al permitir que múltiples múltiplos se marquen simultáneamente. Sin embargo, la eficacia de la paralelización depende de la cantidad de hilos disponibles y cómo se gestiona la carga de trabajo entre ellos.
- **Complejidad**: Aunque el código se vuelve más complejo debido a la gestión de hilos y la sincronización de datos compartidos, Python abstrae muchos de estos detalles, permitiendo una implementación relativamente sencilla de la paralelización.
- **Consideraciones de Rendimiento**: La paralelización introduce overhead debido a la creación de hilos y la sincronización. Para rangos pequeños de`n`, el overhead puede superar los beneficios de la paralelización, por lo que esta técnica es más adecuada para cálculos intensivos a gran escala.

### Conclusión

La paralelización del Criba de Eratóstenes demuestra cómo un algoritmo secuencial puede ser modificado para ejecutarse en paralelo, aprovechando múltiples hilos para mejorar el rendimiento en tareas de cálculo intensivo. Sin embargo, es crucial evaluar el trade-off entre la complejidad del código y la ganancia de rendimiento, especialmente para aplicaciones que se ejecutan en entornos con recursos limitados o para conjuntos de datos más pequeños donde el overhead de la paralelización puede no justificar su uso.